In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [13]:
def read_files(path, filenames):
    """
    Reads all files and returns a dataframe with only the specificed columns
    """
    return pd.concat((pd.read_csv(path + f, skipinitialspace=True) for f in filenames))

def process_options(df_opt, call = True):
    """
    Cleans up column names and add time to live (Ttl) column to the dataframe
    """

    keys = {key: key[key.find("[")+1:key.find("]")][0] + key[key.find("[")+1:key.find("]")][1:].lower()  for key in df_opt.keys()}
    df_opt = df_opt.rename(columns=keys)

    if call:
        keys = {"C_ask": "Ask", "C_bid": "Bid"}
    else:
        keys = {"P_ask": "Ask", "P_bid": "Bid"}
    df_opt = df_opt.rename(columns=keys)

    df_opt["Quote_date"] = pd.to_datetime(df_opt["Quote_date"])
    df_opt["Expire_date"] = pd.to_datetime(df_opt["Expire_date"])
    df_opt["Ttl"] = df_opt.apply(lambda row: (row.Expire_date - row.Quote_date).days, axis = 1)

    df_opt["Moneyness"] = df_opt["Underlying_last"] / df_opt["Strike"]
    
    df_vol = calculate_volatility(df_opt)
    df_vol.info()
    df_opt = pd.merge(df_opt, df_vol, on ="Quote_date", how = "left")

    columns = ["Quote_date", "Expire_date",  "Underlying_last", "Strike", "Moneyness", "Ask", "Bid", "Ttl", "Volatility"]
    df_opt = df_opt[columns]
    df_opt = df_opt[df_opt["Ttl"] != 0]
    return df_opt[columns]

def calculate_volatility(df):
    """
    Calculate underlying volatility from dataset of options
    Returns annualized 90 days moving average volatility
    """
    df_vol = df[["Quote_date", "Underlying_last"]].drop_duplicates()
    df_vol["Volatility"] = np.log(df_vol["Underlying_last"] / df_vol["Underlying_last"].shift()).rolling(90).std()*(252**0.5)
    return df_vol[["Quote_date", "Volatility"]]

def process_rates(df_r):
    """
    Rename rate duration
    """
    df_r["Date"] = pd.to_datetime(df_r["Date"])
    df_r = df_r.rename(columns = {"Date" : "Quote_date", "3 Mo": "R"})
    #rate_keys = {key: key if key == "Date" else int(key.split(" ")[0])*30 if key.split(" ")[1] == "Mo" else int(key.split(" ")[0])*365  for key in df_r.keys()}
    #df_r = df_r.rename(columns=rate_keys)
    columns = ["Quote_date", "R"]
    return df_r[columns]

def combine_opt_r(df_opt, df_r):
    """
    Combines the dataset for options and rates
    """
    #df_opt["R"] = df_opt.apply(lambda row : df_rates[str(min(df_r.drop(["Date"], axis = 1).keys(), key = lambda x:abs(int(x)-row.Ttl)))][row.Quote_date], axis = 1)
    df_opt = pd.merge(df_opt, df_r, on ="Quote_date", how = "left")
    return df_opt

def get_model_dataset(path_opt, filenames_opt, path_r, filenames_r, call = True):
    """
    Wrapper function to extract option data and rates. Returns a combined dataframe
    """
    df_opt = read_files(path_opt, filenames_opt)
    df_r = read_files(path_r, filenames_r)
    df_opt = process_options(df_opt, call)
    df_r = process_rates(df_r)
    df = combine_opt_r(df_opt, df_r)
    return df.dropna() #TODO: Fix handling of nan values

In [14]:
path_opt = "./data/options/"
filenames_opt = ["spx_eod_" + str(year) + (str(month) if month >= 10 else "0"+str(month)) +".txt" for year in range(2022, 2022) for month in range(1, 13)] + ["spx_eod_2022" + (str(month) if month >= 10 else "0"+str(month)) +".txt" for month in range(1, 10)]
#filenames_opt = ["spx_eod_202209.txt"]
path_r = "./data/rates/"
filenames_r = ["yield-curve-rates-2022.csv", "yield-curve-rates-1990-2021.csv"]

df_read = get_model_dataset(path_opt, filenames_opt, path_r, filenames_r, True)
print(df_read)
df_read.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 172733
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Quote_date  192 non-null    datetime64[ns]
 1   Volatility  102 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.5 KB
        Quote_date Expire_date  Underlying_last   Strike  Moneyness    Ask  \
725110  2022-05-09  2022-05-10          3993.26   3100.0   1.288148  893.3   
725111  2022-05-09  2022-05-10          3993.26   3200.0   1.247894  792.9   
725112  2022-05-09  2022-05-10          3993.26   3300.0   1.210079  692.2   
725113  2022-05-09  2022-05-10          3993.26   3400.0   1.174488  592.2   
725114  2022-05-09  2022-05-10          3993.26   3500.0   1.140931  493.0   
...            ...         ...              ...      ...        ...    ...   
1533707 2022-09-30  2026-12-18          3589.70   8800.0   0.407920   31.4   
1533708 2022-09-30

In [26]:
def lag_features(df, features, seq_length):
    """
    Transforms a raw 2D dataframe of option data into 2D dataframe of sequence data
    Last 2 indexes per sequence is bid and ask price
    The len(features)*seq_length features before are sequences of features
    """
    df = df.sort_values(["Expire_date", "Strike", "Ttl"], ascending = [True, True, False])

    for step in range(seq_length):
        for feature in features:
            df[feature + " - " + str(step)] = df[feature].shift(step)
    
    df["Check_strike"] = df["Strike"] == df["Strike"].shift(seq_length-1)
    df["Check_expire"] = df["Expire_date"] == df["Expire_date"].shift(seq_length-1)
    df = df[(df["Check_strike"] == True) & (df["Check_expire"] == True)]
    df = df.drop(["Check_strike", "Check_expire"], axis=1)
    df[["Bid_last", "Ask_last"]] = df[["Bid", "Ask"]]
    return df

def create_train_test(df, features, split_date, seq_length):
    train = lag_features(df[df["Quote_date"] < split_date], features, seq_length).to_numpy()
    test = lag_features(df[df["Quote_date"] >= split_date], features, seq_length).to_numpy()
    print(train[:10][:,-6:])
    train[:, -len(features)*seq_length - 2:], test[:, -len(features)*seq_length - 2:] = min_max_scale(train[:, -len(features)*seq_length-2:], test[:, -len(features)*seq_length-2:])
    #return np.reshape(train_x, (len(train_x), seq_length, len(features))), train_y, np.reshape(test_x, (len(test_x), seq_length, len(features))), test_y
    return train, test #TODO: Move reshaping to modell

def min_max_scale(train, test):
    scaler = MinMaxScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

features = ["Moneyness", "Ttl", "R", "Volatility"]
train, test = create_train_test(df_read, features,  "2022-09-18", 5)

print(train.shape)
print(train)
print(test.shape)
print(train)


[[1.2881483870967743 8.0 0.92 0.23306972873412843 900.1 906.4]
 [1.24789375 8.0 0.92 0.23306972873412843 800.2 806.4]
 [1.210078787878788 8.0 0.92 0.23306972873412843 700.2 706.4]
 [1.1744882352941177 8.0 0.92 0.23306972873412843 600.2 606.4]
 [1.1409314285714287 8.0 0.92 0.23306972873412843 503.1 506.6]
 [1.1092388888888889 8.0 0.92 0.23306972873412843 403.2 406.4]
 [1.0792594594594596 8.0 0.92 0.23306972873412843 303.2 306.8]
 [1.050857894736842 8.0 0.92 0.23306972873412843 203.3 206.9]
 [1.0372103896103897 8.0 0.92 0.23306972873412843 153.6 157.1]
 [1.0305187096774193 8.0 0.92 0.23306972873412843 129.0 132.0]]
(595433, 32)
[[Timestamp('2022-05-16 00:00:00') Timestamp('2022-05-17 00:00:00')
  4009.25 ... 0.007693282197183571 0.21576335786370068
  0.20750958731612387]
 [Timestamp('2022-05-16 00:00:00') Timestamp('2022-05-17 00:00:00')
  4009.25 ... 0.007693282197183571 0.1918162859266007 0.18461450403525845]
 [Timestamp('2022-05-16 00:00:00') Timestamp('2022-05-17 00:00:00')
  4009.25

In [27]:
print(train[:-10][:,-6:])

[[0.021583693424441226 0.0005963029218843171 0.01315789473684209
  0.007693282197183571 0.21576335786370068 0.20750958731612387]
 [0.02064066788222127 0.0005963029218843171 0.01315789473684209
  0.007693282197183571 0.1918162859266007 0.18461450403525845]
 [0.01975479540316616 0.0005963029218843171 0.01315789473684209
  0.007693282197183571 0.16784524294652062 0.161719420754393]
 ...
 [0.0011058482682001474 0.936195587358378 0.8464912280701755
  0.7318589528792989 0.001821799266486085 0.0026214870356590925]
 [0.0012453765445564653 0.9355992844364937 0.8728070175438598
  0.7647292388261464 0.0019176834384064053 0.0027130673687825544]
 [0.0009140320279708743 0.9350029815146094 0.8771929824561404
  0.8866141545670789 0.001821799266486085 0.005483372445767271]]
